<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dist_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install ta

In [0]:
from dist_td3 import *
import os
import multiprocessing as mp
import tensorflow as tf
import numpy as np
import time
import shutil
from multiprocessing import Pool

df = "audpred60.csv"

_ = shutil.copy("/content/drive/My Drive/"+df,"/content")
# sac_one_step.ckpt
saver_path = "td3.ckpt"

restore = True

if restore == True:
  _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".data-00000-of-00001","/content")
  _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".index","/content")
  _ = shutil.copy("/content/drive/My Drive/checkpoint","/content")
  
# if restore == True:
#   _ = shutil.copy("/content/drive/My Drive/model/" + saver_path + ".data-00000-of-00001","/content")
#   _ = shutil.copy("/content/drive/My Drive/model/"+ saver_path + ".index","/content")
#   _ = shutil.copy("/content/drive/My Drive/model/checkpoint","/content")

output_size = 2
window_size = 10
size = 120
num_actor = 8
mem_size = size * num_actor * 1000
  
def actor_work(queues, num):
  sess = tf.InteractiveSession()
  actor = Actor(sess, df, window_size, num, size, output_size, saver_path, restore)
  actor.run(queues, 10, 1000,100,24,n=4)

device='/device:GPU:0'
# device = '/cpu:0'
  
def leaner_work(queues):
        sess = tf.InteractiveSession()
        leaner = Leaner(sess, df, window_size, output_size,mem_size, device, saver_path, restore)
        leaner.leaner(queues)

In [0]:
%cd /content

queue = mp.Queue()
ps = [mp.Process(target=leaner_work, args=(queue,))]
for i in range(num_actor):
  ps.append(mp.Process(target=actor_work, args=(queue,i)))

for p in ps:
    p.start()
    time.sleep(1)

for p in ps:
    p.join()

/content


/content/drive/My Drive/memory.py:127: RuntimeWarning: divide by zero encountered in double_scalars
  max_weight = (p_min * n) ** (-self.PER_b)


action probability =  [0.6974789915966386, 0.3025210084033613, 0.0]
trade accuracy =  0.45689655172413796
epoch: 5, total rewards: 9114.000000, mean rewards: 1838.117647
action probability =  [0.6134453781512605, 0.3865546218487395, 0.0]
trade accuracy =  0.3389830508474576
epoch: 10, total rewards: 2377.000000, mean rewards: 479.394958
action probability =  [0.5882352941176471, 0.4117647058823529, 0.0]
trade accuracy =  0.37606837606837606
epoch: 15, total rewards: -866.000000, mean rewards: -174.655462
action probability =  [0.5798319327731093, 0.42016806722689076, 0.0]
trade accuracy =  0.4017094017094017
epoch: 20, total rewards: 64.000000, mean rewards: 12.907563
action probability =  [0.6218487394957983, 0.37815126050420167, 0.0]
trade accuracy =  0.3879310344827586
epoch: 25, total rewards: 5573.000000, mean rewards: 1123.966387
action probability =  [0.5798319327731093, 0.42016806722689076, 0.0]
trade accuracy =  0.5169491525423728
epoch: 30, total rewards: 7197.000000, mean re

In [0]:
df = pd.read_csv("audpred60.csv")
dat = pd.DataFrame({"Close": df["Close"]})

In [0]:
df["26 ema"] = df["Close"].ewm(span=26).mean()

In [0]:
mp.cpu_count()